In [1]:
import sddk
import pandas as pd
import tabulate
pd.options.display.max_columns = 1000 # to see all columns
import json
import re
import geopandas as gpd
from shapely.geometry import Point
from functools import partial
from shapely.ops import transform
import pyproj
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from scipy.spatial import cKDTree
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

import pickle

import nltk
from nltk.util import bigrams

In [66]:
LIST = gpd.read_parquet("../data/large_data/LIST_contexts.parquet") # point out to your local version...

In [67]:
# remove?
EDH_overlap_all = LIST[(LIST["EDH-ID"].notnull()) & (LIST["EDCS-ID"].notnull())]
EDH_overlap = EDH_overlap_all[~EDH_overlap_all["type_of_inscription_clean"].str.contains("NULL")]

Now we can focus on some EDCS attributes (i.e. firt 28 columns) which might be good predictors of `type_of_inscription_clean` in EDH. First, look at `status_list`:

# Applying the model

In [68]:
# load the model and vectorizer back
classifier = pickle.load(open('../data/large_data/et_model_v2.0.sav', 'rb'))
vectorizer = pickle.load(open('../data/large_data/et_vectorizer_v2.0.sav','rb'))

In [69]:
# we will apply the model to inscriptions which are only in EDCS
EDCS_unique = LIST[(LIST["EDH-ID"].isnull()) & (LIST["EDCS-ID"].notnull())]
EDCS_unique.head(5)

,LIST-ID,EDCS-ID,EDH-ID,trismegistos_uri,pleiades_id,transcription,inscription,clean_text_conservative,clean_text_interpretive_sentence,clean_text_interpretive_word,clean_text_interpretive_word_EDCS,diplomatic_text,province,place,inscr_type,status_notation,inscr_process,status,partner_link,last_update,letter_size,type_of_inscription,work_status,year_of_find,present_location,text_edition,support_objecttype,support_material,support_decoration,keywords_term,people,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,language,language_EDCS,raw_dating,not_after,not_before,Longitude,Latitude,geotemporal?,geometry,withinRE?,large_city_pop,medium_city_pop,small_city_pop,urban_context,urban_context_city,urban_context_pop_est
0,LIST-1,EDCS-31400030,None,None,None,None,Leius,Leius,None,Leius,Leius,None,Achaia,?,None,None,None,None,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,[ ],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,NaN,NaN,False,None,False,NaN,NaN,NaN,None,None,NaN
1,LIST-2,EDCS-55701593,None,None,None,None,Cn(aeus) Atei(us),Cn Atei,None,Cnaeus Ateius,Cnaeus Ateius,None,Achaia,Agios Donatos / Photike,tituli fabricationis,"praenomen et nomen, viri",sigilla impressa,praenomen et nomen; sigilla impressa; tituli...,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,[ ],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,20.506908,39.475976,False,POINT (20.50691 39.47598),True,41221.0,27923.0,1000.0,rural,None,NaN
2,LIST-3,EDCS-55701594,None,None,None,None,Avill(ius),Avill,None,Avillius,Avillius,None,Achaia,Agios Donatos / Photike,tituli fabricationis,"nomen singulare, viri",sigilla impressa,nomen singulare; sigilla impressa; tituli fa...,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,[ ],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,20.506908,39.475976,False,POINT (20.50691 39.47598),True,41221.0,27923.0,1000.0,rural,None,NaN
3,LIST-4,EDCS-55701595,None,None,None,None,L(ucius) Av(illius),L Av,None,Lucius Avillius,Lucius Avillius,None,Achaia,Agios Donatos / Photike,tituli fabricationis,"praenomen et nomen, viri",sigilla impressa,praenomen et nomen; sigilla impressa; tituli...,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,[ ],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,20.506908,39.475976,False,POINT (20.50691 39.47598),True,41221.0,27923.0,1000.0,rural,None,NaN
4,LIST-5,EDCS-55701596,None,None,None,None,Camurius,Camurius,None,Camurius,Camurius,None,Achaia,Agios Donatos / Photike,tituli fabricationis,"nomen singulare, viri",sigilla impressa,nomen singulare; sigilla impressa; tituli fa...,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,[ ],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,20.506908,39.475976,False,POINT (20.50691 39.47598),True,41221.0,27923.0,1000.0,rural,None,NaN


In [70]:
# first create bigrams
def get_bigrams_underscore(inscr_text):
    try:
        inscr_bigrams = [" ".join(el) for el in list(bigrams(inscr_text.split()))]
        inscr_bigrams_ = [bigram.replace(" ", "_") for bigram in inscr_bigrams]
    except:
        inscr_bigrams_ = []
    return inscr_bigrams_

EDCS_unique["bigrams"] = EDCS_unique["clean_text_interpretive_word"].apply(get_bigrams_underscore)

In [71]:
EDCS_unique["status_list"] = EDCS_unique["status"].fillna(value="").apply(lambda x: x.split(";"))
EDCS_unique["material_clean"].replace("NULL", inplace=True)

In [72]:
def combine_status_list_and_bigrams(el_status, el_bigrams, el_material):
    # preprocess status:
    try:
        el_status = eval(el_status)
    except:
        pass
    if isinstance(el_status, list):
        new_el_status = el_status
    elif isinstance(el_status, str):
        new_el_status = [el_status]
    else:
        new_el_status = []
    new_el_status = [el.strip().replace(" ", "_") for el in new_el_status]
    # preprocess material:
    if el_material is None:
        el_material = []
    else:
        el_material = el_material.partition(": ")[0]  #
        el_material = el_material.split(", ")
        el_material = [el.replace(" ", "_").replace("?", "") for el in el_material]
    # combine status, bigrams and material
    new_el = new_el_status + el_bigrams + el_material
    new_el = " ".join(new_el)
    return new_el, new_el_status, el_material

In [73]:
# we need to extract all predictors from the dataframe
# into `status_bigrams_list_apply` (list of lists)
status_bigrams_list_apply = []
status_list = []
material_list = []
for el_status, el_bigrams, el_material in zip(EDCS_unique["status_list"].tolist(), EDCS_unique["bigrams"].tolist(), EDCS_unique["material_clean"].tolist()):
    new_el, new_status, el_material = combine_status_list_and_bigrams(el_status, el_bigrams, el_material)
    status_bigrams_list_apply.append(new_el)
    status_list.append(new_status)
    material_list.extend(el_material)

In [74]:
X_applied = vectorizer.transform(status_bigrams_list_apply)

In [75]:
prediction = classifier.predict(X_applied)
prediction[:10]

array(['identification inscription', 'owner/artist inscription',
       'owner/artist inscription', 'owner/artist inscription',
       'owner/artist inscription', 'owner/artist inscription',
       'votive inscription', 'identification inscription',
       'honorific inscription', 'building/dedicatory inscription'],
      dtype='<U31')

In [76]:
classifier.classes_

array(['acclamation', 'adnuntiatio', 'assignation inscription',
       'boundary inscription', 'building/dedicatory inscription',
       'calendar', 'defixio', 'elogium', 'epitaph',
       'honorific inscription', 'identification inscription', 'label',
       'letter', 'list', 'mile-/leaguestone', 'military diploma',
       'owner/artist inscription', 'prayer', 'private legal inscription',
       'public legal inscription', 'seat inscription',
       'votive inscription'], dtype='<U31')

In [77]:
probs = [np.max(pred) for pred in classifier.predict_proba(X_applied)]
probs[:10]

[0.6666666666666659,
 0.9031917526917528,
 0.8401048945136412,
 0.9031917526917528,
 0.8401048945136412,
 0.9031917526917528,
 0.36523809523809525,
 0.6666666666666659,
 0.62,
 0.3782555555555555]

In [78]:
prediction_probs = [(classifier.classes_[np.argmax(pred)], pred[np.argmax(pred)]) for pred in classifier.predict_proba(X_applied)]
prediction_probs[:10]

[('identification inscription', 0.6666666666666659),
 ('owner/artist inscription', 0.9031917526917528),
 ('owner/artist inscription', 0.8401048945136412),
 ('owner/artist inscription', 0.9031917526917528),
 ('owner/artist inscription', 0.8401048945136412),
 ('owner/artist inscription', 0.9031917526917528),
 ('votive inscription', 0.36523809523809525),
 ('identification inscription', 0.6666666666666659),
 ('honorific inscription', 0.62),
 ('building/dedicatory inscription', 0.3782555555555555)]

In [79]:
len(prediction_probs)

448585

In [80]:
len(prediction_probs)

448585

In [81]:
for threshold in [thres / 100 for thres in range(40, 100, 5)]:
    above_threshold = [tup for tup in prediction_probs if tup[1] >= threshold]
    proportion = len(above_threshold) / len(prediction_probs)
    print("prob. threshold: {0}, proportion of inscr.: {1}".format(threshold, proportion))

prob. threshold: 0.4, proportion of inscr.: 0.9790808876801498
prob. threshold: 0.45, proportion of inscr.: 0.966093382525051
prob. threshold: 0.5, proportion of inscr.: 0.9429115998082861
prob. threshold: 0.55, proportion of inscr.: 0.9070633213326348
prob. threshold: 0.6, proportion of inscr.: 0.8814093204186497
prob. threshold: 0.65, proportion of inscr.: 0.8632388510538694
prob. threshold: 0.7, proportion of inscr.: 0.6805911923046915
prob. threshold: 0.75, proportion of inscr.: 0.6581784945996857
prob. threshold: 0.8, proportion of inscr.: 0.5933189919413266
prob. threshold: 0.85, proportion of inscr.: 0.5478627238984808
prob. threshold: 0.9, proportion of inscr.: 0.5202313942731032
prob. threshold: 0.95, proportion of inscr.: 0.4698173144443082


In [82]:
threshold_results = []
for threshold in [thres / 100 for thres in range(30, 100, 2)]:
    above_threshold = [tup for tup in prediction_probs if tup[1] >= threshold]
    proportion = len(above_threshold) / len(prediction_probs)
    threshold_results.append({"threshold (=/>)" : threshold, "proportion" : np.round(proportion, 2), "N" : len(above_threshold)})
model_results_df = pd.DataFrame(threshold_results)
model_results_df

,threshold (=/>),proportion,N
0,0.30,0.99,445898
1,0.32,0.99,445147
2,0.34,0.99,444183
3,0.36,0.99,443094
4,0.38,0.98,440821
5,0.40,0.98,439201
6,0.42,0.97,436929
7,0.44,0.97,434828
8,0.46,0.96,431418
9,0.48,0.95,427592


In [83]:
#print(model_results_df.to_latex(index=False))

In [84]:
#model_results_df.to_csv("../data/classifier_EDCS_results.csv")

# Combining the data back

In [85]:
EDCS_unique["type_of_inscription_auto"] = prediction
EDCS_unique["type_of_inscription_auto_prob"] = probs

In [86]:
EDCS_unique.head(5)

,LIST-ID,EDCS-ID,EDH-ID,trismegistos_uri,pleiades_id,transcription,inscription,clean_text_conservative,clean_text_interpretive_sentence,clean_text_interpretive_word,clean_text_interpretive_word_EDCS,diplomatic_text,province,place,inscr_type,status_notation,inscr_process,status,partner_link,last_update,letter_size,type_of_inscription,work_status,year_of_find,present_location,text_edition,support_objecttype,support_material,support_decoration,keywords_term,people,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,language,language_EDCS,raw_dating,not_after,not_before,Longitude,Latitude,geotemporal?,geometry,withinRE?,large_city_pop,medium_city_pop,small_city_pop,urban_context,urban_context_city,urban_context_pop_est,bigrams,status_list,type_of_inscription_auto,type_of_inscription_auto_prob
0,LIST-1,EDCS-31400030,None,None,None,None,Leius,Leius,None,Leius,Leius,None,Achaia,?,None,None,None,None,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,[ ],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,NaN,NaN,False,None,False,NaN,NaN,NaN,None,None,NaN,[],[],identification inscription,0.666667
1,LIST-2,EDCS-55701593,None,None,None,None,Cn(aeus) Atei(us),Cn Atei,None,Cnaeus Ateius,Cnaeus Ateius,None,Achaia,Agios Donatos / Photike,tituli fabricationis,"praenomen et nomen, viri",sigilla impressa,praenomen et nomen; sigilla impressa; tituli...,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,[ ],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,20.506908,39.475976,False,POINT (20.50691 39.47598),True,41221.0,27923.0,1000.0,rural,None,NaN,[Cnaeus_Ateius],"[praenomen et nomen, sigilla impressa, tit...",owner/artist inscription,0.903192
2,LIST-3,EDCS-55701594,None,None,None,None,Avill(ius),Avill,None,Avillius,Avillius,None,Achaia,Agios Donatos / Photike,tituli fabricationis,"nomen singulare, viri",sigilla impressa,nomen singulare; sigilla impressa; tituli fa...,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,[ ],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,20.506908,39.475976,False,POINT (20.50691 39.47598),True,41221.0,27923.0,1000.0,rural,None,NaN,[],"[nomen singulare, sigilla impressa, tituli...",owner/artist inscription,0.840105
3,LIST-4,EDCS-55701595,None,None,None,None,L(ucius) Av(illius),L Av,None,Lucius Avillius,Lucius Avillius,None,Achaia,Agios Donatos / Photike,tituli fabricationis,"praenomen et nomen, viri",sigilla impressa,praenomen et nomen; sigilla impressa; tituli...,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,[ ],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,20.506908,39.475976,False,POINT (20.50691 39.47598),True,41221.0,27923.0,1000.0,rural,None,NaN,[Lucius_Avillius],"[praenomen et nomen, sigilla impressa, tit...",owner/artist inscription,0.903192
4,LIST-5,EDCS-55701596,None,None,None,None,Camurius,Camurius,None,Camurius,Camurius,None,Achaia,Agios Donatos / Photike,tituli fabricationis,"nomen singulare, viri",sigilla impressa,nomen singulare; sigilla impressa; tituli fa...,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,[ ],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,20.506908,39.475976,False,POINT (20.50691 39.47598),True,41221.0,27923.0,1000.0,rural,None,NaN,[],"[nomen singulare, sigilla impressa, tituli...",owner/artist inscription,0.840105


In [87]:
# prepare last part of the data - inscriptions we have only in EDH
# in this case, we can also use the information we already have
EDH_unique = LIST[(LIST["EDH-ID"].notnull()) & (LIST["EDCS-ID"].isnull())]
EDH_unique["type_of_inscription_auto"] = EDH_unique["type_of_inscription_clean"]
EDH_unique["type_of_inscription_auto_prob"] = [1] * len(EDH_unique)

In [88]:
EDH_overlap_all = LIST[(LIST["EDH-ID"].notnull()) & (LIST["EDCS-ID"].notnull())]
EDH_overlap_all["type_of_inscription_auto"] = EDH_overlap_all["type_of_inscription_clean"]
EDH_overlap_all["type_of_inscription_auto_prob"] = [1] * len(EDH_overlap_all)

In [89]:
print("the dataset we used for training and testing (+NULLs): ", EDH_overlap_all.shape)
print("inscriptions in EDH only: ", EDH_unique.shape)
print("inscriptions in EDCS only, we wanted to classify: ", EDCS_unique.shape)

the dataset we used for training and testing (+NULLs):  (76583, 69)
inscriptions in EDH only:  (3316, 69)
inscriptions in EDCS only, we wanted to classify:  (448585, 71)


In [90]:
# combine together
LIST_autotypes = pd.concat([EDH_overlap_all, EDH_unique, EDCS_unique])
LIST_autotypes.head()

,LIST-ID,EDCS-ID,EDH-ID,trismegistos_uri,pleiades_id,transcription,inscription,clean_text_conservative,clean_text_interpretive_sentence,clean_text_interpretive_word,clean_text_interpretive_word_EDCS,diplomatic_text,province,place,inscr_type,status_notation,inscr_process,status,partner_link,last_update,letter_size,type_of_inscription,work_status,year_of_find,present_location,text_edition,support_objecttype,support_material,support_decoration,keywords_term,people,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,language,language_EDCS,raw_dating,not_after,not_before,Longitude,Latitude,geotemporal?,geometry,withinRE?,large_city_pop,medium_city_pop,small_city_pop,urban_context,urban_context_city,urban_context_pop_est,type_of_inscription_auto,type_of_inscription_auto_prob,bigrams,status_list
448585,LIST-448586,EDCS-24900077,HD056163,https://www.trismegistos.org/text/177366,570485,Q(uinto) Caecilio C(ai) f(ilio) Metelo / imper...,Q(uinto) Caecilio C(ai) f(ilio) Metel(l)o / im...,Q Caecilio C f Metelo imperatori Italici quei ...,Quinto Caecilio Cai filio Metelo imperatori It...,Quinto Caecilio Cai filio Metelo imperatori It...,Quinto Caecilio Cai filio Metello imperatori I...,Q CAECILIO C F METELO / IMPERATORI ITALICI / Q...,Achaia,Agia Triada / Merbaka / Midea,tituli honorarii,"officium/professio, ordo senatorius, tria nomi...",None,officium/professio; ordo senatorius; tituli ...,http://db.edcs.eu/epigr/partner.php?s_language...,2011-11-11,None,honorific inscription,no image,NaN,None,\n Quinto Caecilio Cai filio Metelo imperatori...,None,NaN,1000.0,69.0,[ ],honorific inscription,False,NaN,NaN,NaN,NULL,NULL,False,Achaia,False,Greece,False,Midea,False,Pelopónissos,False,Midhéa,False,NULL,False,L,None,-68 to -68,-68.0,NaN,22.8412,37.6498,True,POINT (22.84120 37.64980),True,49319.0,9938.0,1000.0,rural,None,NaN,honorific inscription,1.0,NaN,NaN
448586,LIST-448587,EDCS-03700724,HD052964,https://www.trismegistos.org/text/121715,531064,Fortissimo et piis/simo Caesari d(omino) n(ost...,Fortissimo et Piis/simo Caesari d(omino) n(ost...,Fortissimo et piissimo Caesari d n Gal Val P F...,Fortissimo et piissimo Caesari domino nostro G...,Fortissimo et piissimo Caesari domino nostro G...,Fortissimo et Piissimo Caesari domino nostro G...,FORTISSIMO ET PIIS / SIMO CAESARI D N / GAL VA...,Achaia,Agios Athanasios / Photike,tituli honorarii,"Augusti/Augustae, ordo equester, tria nomina",litterae erasae,Augusti/Augustae; litterae erasae; ordo eque...,http://db.edcs.eu/epigr/partner.php?s_language...,2014-09-16,3-5.3 cm,honorific inscription,checked with photo,NaN,Fragma Kalama,\n Fortissimo et piissimo Caesari domino nostr...,57,NaN,1000.0,69.0,"[ { ""persname"": { ""name"": [ { ""@type"": ""nomen""...",honorific inscription,False,99.0,67.0,67.0,NULL,statue base,False,Epirus,False,Greece,False,Photike,False,Ípeiros,False,Paramythía,False,{Agios Athanasios},False,L,None,309 to 313,309.0,313.0,20.7668,39.4512,True,POINT (20.76680 39.45120),True,41221.0,27923.0,1000.0,rural,None,NaN,honorific inscription,1.0,NaN,NaN
448587,LIST-448588,EDCS-13800065,HD017714,https://www.trismegistos.org/text/177100,570049,Italicei / quei Aegei negotiantur / P(ublium) ...,Italicei / quei Aegei negotiantur / P(ublium) ...,Italicei quei Aegei negotiantur P Rutilium P f...,Italicei quei Aegei negotiantur Publium Rutili...,Italicei quei Aegei negotiantur Publium Rutili...,Italicei quei Aegei negotiantur Publium Rutili...,ITALICEI / QVEI AEGEI NEGOTIANTVR / P RVTILIVM...,Achaia,Aigio / Egio / Aiyion / Aegeum,tituli honorarii,"officium/professio, ordo senatorius, tria nomi...",None,officium/professio; ordo senatorius; tituli ...,http:

In [91]:
LIST_autotypes.drop(["bigrams", "status_list"], axis=1, inplace=True)

In [92]:
LIST_autotypes.shape

(528484, 69)

In [93]:
LIST_autotypes.head(5)

,LIST-ID,EDCS-ID,EDH-ID,trismegistos_uri,pleiades_id,transcription,inscription,clean_text_conservative,clean_text_interpretive_sentence,clean_text_interpretive_word,clean_text_interpretive_word_EDCS,diplomatic_text,province,place,inscr_type,status_notation,inscr_process,status,partner_link,last_update,letter_size,type_of_inscription,work_status,year_of_find,present_location,text_edition,support_objecttype,support_material,support_decoration,keywords_term,people,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,language,language_EDCS,raw_dating,not_after,not_before,Longitude,Latitude,geotemporal?,geometry,withinRE?,large_city_pop,medium_city_pop,small_city_pop,urban_context,urban_context_city,urban_context_pop_est,type_of_inscription_auto,type_of_inscription_auto_prob
448585,LIST-448586,EDCS-24900077,HD056163,https://www.trismegistos.org/text/177366,570485,Q(uinto) Caecilio C(ai) f(ilio) Metelo / imper...,Q(uinto) Caecilio C(ai) f(ilio) Metel(l)o / im...,Q Caecilio C f Metelo imperatori Italici quei ...,Quinto Caecilio Cai filio Metelo imperatori It...,Quinto Caecilio Cai filio Metelo imperatori It...,Quinto Caecilio Cai filio Metello imperatori I...,Q CAECILIO C F METELO / IMPERATORI ITALICI / Q...,Achaia,Agia Triada / Merbaka / Midea,tituli honorarii,"officium/professio, ordo senatorius, tria nomi...",None,officium/professio; ordo senatorius; tituli ...,http://db.edcs.eu/epigr/partner.php?s_language...,2011-11-11,None,honorific inscription,no image,NaN,None,\n Quinto Caecilio Cai filio Metelo imperatori...,None,NaN,1000.0,69.0,[ ],honorific inscription,False,NaN,NaN,NaN,NULL,NULL,False,Achaia,False,Greece,False,Midea,False,Pelopónissos,False,Midhéa,False,NULL,False,L,None,-68 to -68,-68.0,NaN,22.8412,37.6498,True,POINT (22.84120 37.64980),True,49319.0,9938.0,1000.0,rural,None,NaN,honorific inscription,1.0
448586,LIST-448587,EDCS-03700724,HD052964,https://www.trismegistos.org/text/121715,531064,Fortissimo et piis/simo Caesari d(omino) n(ost...,Fortissimo et Piis/simo Caesari d(omino) n(ost...,Fortissimo et piissimo Caesari d n Gal Val P F...,Fortissimo et piissimo Caesari domino nostro G...,Fortissimo et piissimo Caesari domino nostro G...,Fortissimo et Piissimo Caesari domino nostro G...,FORTISSIMO ET PIIS / SIMO CAESARI D N / GAL VA...,Achaia,Agios Athanasios / Photike,tituli honorarii,"Augusti/Augustae, ordo equester, tria nomina",litterae erasae,Augusti/Augustae; litterae erasae; ordo eque...,http://db.edcs.eu/epigr/partner.php?s_language...,2014-09-16,3-5.3 cm,honorific inscription,checked with photo,NaN,Fragma Kalama,\n Fortissimo et piissimo Caesari domino nostr...,57,NaN,1000.0,69.0,"[ { ""persname"": { ""name"": [ { ""@type"": ""nomen""...",honorific inscription,False,99.0,67.0,67.0,NULL,statue base,False,Epirus,False,Greece,False,Photike,False,Ípeiros,False,Paramythía,False,{Agios Athanasios},False,L,None,309 to 313,309.0,313.0,20.7668,39.4512,True,POINT (20.76680 39.45120),True,41221.0,27923.0,1000.0,rural,None,NaN,honorific inscription,1.0
448587,LIST-448588,EDCS-13800065,HD017714,https://www.trismegistos.org/text/177100,570049,Italicei / quei Aegei negotiantur / P(ublium) ...,Italicei / quei Aegei negotiantur / P(ublium) ...,Italicei quei Aegei negotiantur P Rutilium P f...,Italicei quei Aegei negotiantur Publium Rutili...,Italicei quei Aegei negotiantur Publium Rutili...,Italicei quei Aegei negotiantur Publium Rutili...,ITALICEI / QVEI AEGEI NEGOTIANTVR / P RVTILIVM...,Achaia,Aigio / Egio / Aiyion / Aegeum,tituli honorarii,"officium/professio, ordo senatorius, tria nomi...",None,officium/professio; ordo senatorius; tituli ...,http://db.edcs.eu/epigr/partner.php?s_lan

In [94]:
len(LIST_autotypes[LIST_autotypes["type_of_inscription_auto_prob"]>=0.6])

475286

In [95]:
len(LIST_autotypes[LIST_autotypes["type_of_inscription_auto"]=="NULL"])

20388

In [96]:
def apply_threshold(inscr_type, prob):
    if (prob < 0.6) or (inscr_type == "NULL"):
        return None
    else:
        return inscr_type
LIST_autotypes["type_of_inscription_auto"] =  LIST_autotypes.apply(lambda row: apply_threshold(row["type_of_inscription_auto"], row["type_of_inscription_auto_prob"]), axis=1)

In [97]:
def prob_to_zero(inscr_type, prob):
    if inscr_type == None:
        return 0
    else:
        return prob
LIST_autotypes["type_of_inscription_auto_prob"] =  LIST_autotypes.apply(lambda row: prob_to_zero(row["type_of_inscription_auto"], row["type_of_inscription_auto_prob"]), axis=1)

In [98]:
len(LIST_autotypes[LIST_autotypes["type_of_inscription_auto"].notnull()])

454898

# Save

In [100]:
LIST_autotypes.to_parquet("../data/large_data/LIST_v0-2.parquet")